# Mobile Customer Churn

In this Portfolio task you will work with some (fake but realistic) data on Mobile Customer Churn.  Churn is where
a customer leaves the mobile provider.   The goal is to build a simple predictive model to predict churn from available features. 

The data was generated (by Hume Winzar at Macquarie) based on a real dataset provided by Optus.  The data is simulated but the column headings are the same. (Note that I'm not sure if all of the real relationships in this data are preserved so you need to be cautious in interpreting the results of your analysis here).  

The data is provided in file `MobileCustomerChurn.csv` and column headings are defined in a file `MobileChurnDataDictionary.csv` (store these in the `files` folder in your project).

Your high level goal in this notebook is to try to build and evaluate a __predictive model for churn__ - predict the value of the CHURN_IND field in the data from some of the other fields.  Note that the three `RECON` fields should not be used as they indicate whether the customer reconnected after having churned. 

__Note:__ you are not being evaluated on the _accuracy_ of the model but on the _process_ that you use to generate it.  You can use a simple model such as Logistic Regression for this task or try one of the more advanced methods covered in recent weeks.  Explore the data, build a model using a selection of features and then do some work on finding out which features provide the most accurate results.  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
churn = pd.read_csv("MobileCustomerChurn.csv", na_values=["NA", "#VALUE!"], index_col='INDEX')
churn.head()

,CUST_ID,ACCOUNT_TENURE,ACCT_CNT_SERVICES,AGE,CFU,SERVICE_TENURE,PLAN_ACCESS_FEE,BYO_PLAN_STATUS,PLAN_TENURE,MONTHS_OF_CONTRACT_REMAINING,...,CONTRACT_STATUS,PREV_CONTRACT_DURATION,HANDSET_USED_BRAND,CHURN_IND,MONTHLY_SPEND,COUNTRY_METRO_REGION,STATE,RECON_SMS_NEXT_MTH,RECON_TELE_NEXT_MTH,RECON_EMAIL_NEXT_MTH
INDEX,,,,,,,,,,,,,,,,,,,,,
1,1,46,1,30.0,CONSUMER,46,54.54,NON BYO,15,0,...,OFF-CONTRACT,24,SAMSUNG,1,61.40,COUNTRY,WA,NaN,NaN,NaN
2,2,60,3,55.0,CONSUMER,59,54.54,NON BYO,5,0,...,OFF-CONTRACT,24,APPLE,1,54.54,METRO,NSW,NaN,NaN,NaN
3,5,65,1,29.0,CONSUMER,65,40.90,BYO,15,0,...,OFF-CONTRACT,12,APPLE,1,2.50,COUNTRY,WA,NaN,NaN,NaN
4,6,31,1,51.0,CONSUMER,31,31.81,NON BYO,31,0,...,OFF-CONTRACT,24,APPLE,1,6.48,COUNTRY,VIC,NaN,NaN,NaN
5,8,95,1,31.0,CONSUMER,95,54.54,NON BYO,0,0,...,OFF-CONTRACT,24,APPLE,1,100.22,METRO,NSW,NaN,NaN,NaN


In [3]:
#logistical regression (week 6) Test and evaluate (PREDICT CHURN IND FROM OTHER FIELDS)
#Drop all non-numerical values before using algorithm

In [4]:
churn = churn.dropna() #DROP NA VALUES 
churn.isna().sum() #check if dropped

CUST_ID                         0
ACCOUNT_TENURE                  0
ACCT_CNT_SERVICES               0
AGE                             0
CFU                             0
SERVICE_TENURE                  0
PLAN_ACCESS_FEE                 0
BYO_PLAN_STATUS                 0
PLAN_TENURE                     0
MONTHS_OF_CONTRACT_REMAINING    0
LAST_FX_CONTRACT_DURATION       0
CONTRACT_STATUS                 0
PREV_CONTRACT_DURATION          0
HANDSET_USED_BRAND              0
CHURN_IND                       0
MONTHLY_SPEND                   0
COUNTRY_METRO_REGION            0
STATE                           0
RECON_SMS_NEXT_MTH              0
RECON_TELE_NEXT_MTH             0
RECON_EMAIL_NEXT_MTH            0
dtype: int64

In [5]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
train, test = train_test_split(churn, test_size=0.2, random_state=42) #more training, less testing
print(train.shape)   #fixing random_state gives same result
print(test.shape)

(22692, 21)
(5674, 21)


In [13]:
x_train = train.drop(['BYO_PLAN_STATUS','CFU','CONTRACT_STATUS','HANDSET_USED_BRAND','COUNTRY_METRO_REGION','STATE','RECON_SMS_NEXT_MTH','RECON_TELE_NEXT_MTH','RECON_EMAIL_NEXT_MTH'],axis=1)
y_train = train['CHURN_IND']

x_test = test.drop(['BYO_PLAN_STATUS','CFU','CONTRACT_STATUS','HANDSET_USED_BRAND','COUNTRY_METRO_REGION','STATE','RECON_SMS_NEXT_MTH','RECON_TELE_NEXT_MTH','RECON_EMAIL_NEXT_MTH'],axis=1)
y_test = test['CHURN_IND']

In [15]:
#training
lr = linear_model.LinearRegression() #CHANGE TO LOGISTIC REGRESSION (set('CHURN_NID') only has 0 classes when requires >=2???)
lr.fit(x_train,y_train)

LinearRegression()

In [16]:
#Testing
y_pred = lr.predict(x_test)
y_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [17]:
print(y_test[:5])
print(y_pred[:5])

INDEX
34099    0
25554    0
39929    0
17667    0
41313    0
Name: CHURN_IND, dtype: int64
[0. 0. 0. 0. 0.]


In [18]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

mse_lr = mean_squared_error(y_test,y_pred)
r2 = r2_score(y_test,y_pred)

print("MSE:", mse_lr) 
print("R Squared:", r2)

MSE: 0.0
R Squared: 1.0
